In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
import subprocess, os, shutil, warnings, csv, json
from Stats import *
from skopt import gp_minimize, Optimizer
from skopt.space import Real
from skopt.utils import dump
import multiprocessing as mp

from Circuit_Properties import *
from Stats import *
from Functions import *
from HexProperties import *
from rstools import RSoftUserFunction, RSoftCircuit
'''
Set RSoft parameters, dump them in a json file and generate the RSoft circuit
'''

name = "MCF_Test"
sym = {}

# set fibre parameters to be used in template file
length = 1000
Corediam = 8.2
Claddiam = 125
Core_sep = 60
taper_ratio = 1
core_num = 7 # this must be an odd number!!
Core_delta = 0.01
background_index = 1.456
delta = 0.012

# checking to see if the cores fully populate the hex grid
number_rows(core_num)

# simulation stuff
Dx = 0.1
Dy = Dx
Dz = 0.2
Phase = 0
boundary_gap_x = 10
boundary_gap_y = boundary_gap_x
boundary_gap_z = 0
bpm_pathway = 1
bpm_pathway_monitor = bpm_pathway
sym_tool = 'ST_BEAMPROP'
width = 5
height = width
H = height
grid_uniform = 0
eim = 0
polarization = 0
free_space_wavelength = 1
k0 = 2*np.pi / free_space_wavelength
slice_display_mode = 'DISPLAY_CONTOURMAPXZ'
slice_position_z = 100

# launch properties
launch_tilt = 1
launch_port = 1
launch_align_file = 1
launch_mode = 0
launch_mode_radial = 1
launch_normalization = 1
grid_size = 'Dx'
grid_size_y = 'Dy'
step_size = 'Dz'

structure = 'STRUCT_FIBER'

#################################################################################
'''
Json Derulo
'''
# dump variable parameters in json file
with open("variable_paras.json", "w") as g:
        json.dump({"Name": name,
                   "Length": length,
                   "Corediam": Corediam,
                   "Claddiam": Claddiam,
                   "Core_sep": Core_sep,
                   "taper_ratio": taper_ratio,
                   "core_num": core_num,
                   "Core_delta": Core_delta,
                   "background_index": background_index,
                   "delta": delta
        }, g)

# dump fixed parameters in json file
with open("fibre_prop.json", "w") as f:
        json.dump({"Dx": Dx,
                   "Dy": Dy,
                   "Dz": Dz,
                   "Phase": Phase,
                   "boundary_gap_x": boundary_gap_x,
                   "boundary_gap_y": boundary_gap_y,
                   "boundary_gap_z": boundary_gap_z,
                   "bpm_pathway": bpm_pathway,
                   "bpm_pathway_monitor": bpm_pathway_monitor,
                   "sym_tool": sym_tool,
                   "width": width,
                   "height": height,
                   "H": height,
                   "grid_uniform": grid_uniform,
                   "eim": eim,
                   "polarization": polarization,
                   "free_space_wavelength": free_space_wavelength,
                   "k0": k0,
                   "slice_display_mode": slice_display_mode,
                   "slice_position_z": slice_position_z,
                   "launch_tilt": launch_tilt,
                   "launch_port": launch_port,
                   "launch_align_file": launch_align_file,
                   "launch_mode": launch_mode,
                   "launch_mode_radial": launch_mode_radial,
                   "launch_normalization": launch_normalization,
                   "grid_size": grid_size,
                   "grid_size_y": grid_size_y,
                   "step_size": step_size,
                   "structure": structure
                   }, f)

# properties of the structure (units of um)
with open("variable_paras.json","r") as g:
    param_v = json.load(g)
with open("fibre_prop.json", "r") as f:
    params = json.load(f)

# initialise prior space
para_space = {
    "Corediam": (1.0,40.0),
    "Length": (10.0, 1000.0),
    "Core_index": (background_index, 2.0)
}

# dump prior space
with open("prior_space.json", "w") as write:
    json.dump(para_space, write)
#################################################################################

# assign parameters to RSoft circuit
for key_v in param_v:
    sym[key_v] = param_v[key_v]
for key in params:
    sym[key] = params[key]

# creating the design file, load the settings and add symbols
c = RSoftCircuit()
for key in sym:
    c.set_symbol(key,sym[key])

In [2]:
''' 
Generating the positional coordinates for each fibre 
'''

if core_num % 2 == 0:
    raise ValueError(f"The number of cores must be odd to perfectly fit inside the hex grid. Received:{core_num}")
    
row_numbers = [number_rows(core_num)]
'''
TO DO: add options for the distribution of cores, make this a function that takes a string ("Hex", "Pent", "Circle" etc)
'''
for idx, row_num in enumerate(row_numbers):
    hcoord, vcoord = generate_hex_grid(row_num, sym["Core_sep"])

In [3]:
''' 
Generating the segments and assigning the positional coordinates found in 
the previous sections 
'''

# Add the outer cladding segment
cladding_beg_dims = (sym['Claddiam'] / sym['taper_ratio'], sym['Claddiam'] / sym['taper_ratio'])
cladding_end_dims = (sym['Claddiam'] , sym['Claddiam'])

# Add the individual core segments
core_beg_dims = ('Corediam','Corediam')
core_end_dims = ('Corediam','Corediam')

if core_num == 1: 
    cladding = c.add_segment(
        position=(0, 0, 0),
        offset=(0, 0, 'Length'),
        dimensions=core_beg_dims,
        dimensions_end=core_end_dims
    )
    cladding.set_name("MMF Cladding")
else:
    cladding = c.add_segment(
        position=(0, 0, 0),
        offset=(0, 0, 'Length'),
        dimensions=core_beg_dims,
        dimensions_end=core_end_dims
    )
    cladding.set_name("MMF Cladding")
    
    core_name = [f"core_{n+1:02}" for n in range(sym['core_num'])]

    for j, (x, y) in enumerate(zip(hcoord, vcoord)):
        core = c.add_segment(
                    position=(x/sym['taper_ratio'],y/sym['taper_ratio'],0), 
                    offset=(x,y,'Length'), 
                    dimensions = core_beg_dims, 
                    dimensions_end = core_end_dims
                    )
        core.set_name(core_name[j])

# write to file
c.write(f"{name}.ind")
'''
This is the hacking part which adds all of the pathways, then the monitors and then the 
launch fields. They are done individually to account for formatting issues that may exist
within RSoft's .ind files.
'''
# extract the number of cores from the template file
# core_num = int(Extract_params("core_num "))

AddHack(name, int(core_num), Corediam, Core_delta)

In [ ]:
'''
Multiprocessing must be run outside of a Jupyter cell or else it will silently fail/infinitely loop on the first batch
'''
subprocess.run(["python", "Multiprocessing.py"], check=True)

In [17]:
# Read the multiprocessing csv log into a DataFrame
df = pd.read_csv("best_params_log.csv", header=None)

param_columns = [prior_name for prior_name, (_,_) in para_space.items()]
print(param_columns)
for col in param_columns:
    if col in df.columns:
        print(f"{col}: {df[col].iloc[-1]}")


['Corediam', 'Length', 'Core_index']


In [6]:
# Iteration = 0
# def RunRsoft(params, skip=False): 
#     global Iteration
#     Iteration += 1

#     if isinstance(params, (list, tuple)):
#         core_diam = params[0]
#     else:
#         core_diam = params

#     print(f"[Iteration {Iteration}] core_diam = {core_diam:.2f}")
    
#     ###########################################################################################
#     '''
#     The purpose of skipping the simulation is to test if the plotting works within the function.
#     '''
#     if not skip: 
#         '''
#         Manual setup to loop through a list of values
#         Runs the terminal line that will initiate RSoft. 
#         All output files will appear in a subfolder on the Desktop (windows)
#         '''

#         # initialise results dictionary
#         results = {}
#         l = 0

#         # open and read the generated .ind file
#         with open(f"{name}.ind", "r") as r:
#             lines = r.readlines()

#             # loop through list of parameters
#             '''
#             To add: make this universal so any list of parameters can be looped through, not just core diameter
#             '''
#             modified_line = []
#             for line in lines:
#                 if line.strip().startswith("Corediam ="):
#                     rep_line = f"Corediam = {core_diam}\n"
#                     modified_line.append(rep_line)
#                 else:
#                     modified_line.append(line)

#             # generate separate .ind file based on parameters
#             with open(f"MCF_{core_diam}.ind", "w") as out:
#                 out.writelines(modified_line)

#             # This runs the beamprop simulation and also allocates the results in their own folder on the desktop
#             # Simulation tags
#             filename = f"MCF_{core_diam}.ind"
#             prefix = f"prefix=Corediam_{core_diam}"
#             Folder_name = f"CoreSim_{core_diam}"

#             # Creates results folder on Desktop
#             user_home = os.path.expanduser("~")
#             try:
#                 desktop_path = os.path.join(user_home, "Desktop")
#             except Exception:
#                 desktop_path = os.getcwd()
#                 warnings.warn("Results are not placed in predefined folder! Prepare for organised chaos!")
                
#             results_folder_root = os.path.join(desktop_path,"Results")
#             os.makedirs(results_folder_root, exist_ok=True)

#             # Creates sub-folder within results folder to better allocate results
#             results_folder = os.path.join(results_folder_root,Folder_name)
#             os.makedirs(results_folder, exist_ok = True)

#             # Run RSoft simulation
#             subprocess.run(["bsimw32", filename, prefix, "wait=0"], check=True)

#             '''
#             To do: make this test metric a choice. Incorporate other metric and make it an 
#             option to choose between metrics to test.
#             '''
#             # read pathway monitor file
#             uf.read(f'Corediam_{core_diam}.mon')
#             (x,y) = uf.get_arrays()
#             # x is the fibre length
#             # y is the pathway monitor power

#             results[l] = y
#             l += 1

#             for file in os.listdir():
#                 if file.startswith(f"Corediam_{core_diam}") or file == filename:
#                     shutil.move(file, os.path.join(results_folder, file))

#         with open("Throughput_results.csv", mode = "w", newline="") as file:
#             writer = csv.writer(file)
#             header = ["x"] + [f"{core_diam:.2f}"]
#             writer.writerow(header)

#             for i in range(len(x)):
#                 row = [x[i]] + [results[j][i] for j in range(l)]
#                 writer.writerow(row)

#     elif skip:
#         warnings.warn('Prior space will change, use plot only for validation purposes!')
#     ###########################################################################################

#     df = pd.read_csv('Throughput_results.csv')

#     col_names = f"{core_diam:.2f}"
#     if col_names not in df.columns:
#         raise KeyError(f"Column '{col_names}' not present in CSV file.")
    
#     average = df[col_names].mean()
#     return -average

In [7]:
# import logging
# # Iteration = 0
# def RunRsoft(params, skip=False): 
#     # global Iteration
#     # Iteration += 1

#     param_dict = {dim.name: val for dim, val in zip(para_space, params)}
#     # print(param_dict)
#     # print(f"[Iteration {Iteration}]: " + ", ".join(f"{param} = {val:.4f}" for param, val in param_dict.items()))
#     logging.basicConfig(level=logging.INFO)
#     logger = logging.getLogger(__name__)
#     logger.info(f"Running iteration with params: {param_dict}")
#     ###########################################################################################
#     '''
#     The purpose of skipping the simulation is to test if the plotting works within the function.
#     '''
#     if not skip: 
#         '''
#         Manual setup to loop through a list of values
#         Runs the terminal line that will initiate RSoft. 
#         All output files will appear in a subfolder on the Desktop (windows)
#         '''
#         name_tag = "_".join(f"{key}_{val:.4f}" for key, val in param_dict.items())

#         filename = f"MCF_{name_tag}.ind"
#         prefix   = f"prefix={name_tag}"
#         folder   = f"Sim_{name_tag}"

#         # # Read original template
#         # with open(f"{name}.ind", "r") as r:
#         #     lines = r.readlines()
#         #     with open(filename, "w") as mod_file:
#         #         while has_another_line(r):
#         #             readString = r.readline()
#         #             if readString.startswith("segment"):
#         #                 mod_file.write(readString)
#         #                 mod_file.write("\tbegin.delta = " + str(core_index) + "-background_index\n")
#         #                 mod_file.write("\tend.delta = " + str(core_index) + "-background_index\n")
#         #             else:
#         #                 mod_file.write(readString)

#         # # Replace parameters
#         # modified_lines = []
#         # for line in lines:
#         #     line_strip = line.strip()
#         #     replaced = False
#         #     for param, val in param_dict.items():
#         #         if param == "Core_index":
#         #             continue # already replaced the index value above
#         #         if line_strip.startswith(f"{param} ="):
#         #             modified_lines.append(f"{param} = {val:.4f}\n")
#         #             replaced = True
#         #             break
#         #     if not replaced:
#         #         modified_lines.append(line)
#         ##########################################################################################
#         # Read original template
#         with open(f"{name}.ind", "r") as r:
#             lines = r.readlines()

#         # Construct symbolic delta expression
#         delta_expr = param_dict["Core_index"] - background_index

#         # Build the updated lines
#         modified_lines = []
#         for line in lines:
#             line_strip = line.strip()
#             replaced = False

#             # Insert delta after segment start
#             if line_strip.startswith("segment"):
#                 modified_lines.append(line)
#                 modified_lines.append(f"\tbegin.delta = {delta_expr}\n")
#                 modified_lines.append(f"\tend.delta = {delta_expr}\n")
#                 continue

#             # Skip replacing Core_index directly (if already covered elsewhere)
#             for param, val in param_dict.items():
#                 if param == "Core_index":
#                     continue
#                 if line_strip.startswith(f"{param} ="):
#                     modified_lines.append(f"{param} = {val:.4f}\n")
#                     replaced = True
#                     break

#             if not replaced:
#                 modified_lines.append(line)

#         # Write the final .ind file with symbolic delta expression
#         with open(filename, "w") as out:
#             out.writelines(modified_lines)

#         ##########################################################################################
#         #  # Write new .ind file
#         # with open(filename, "w") as out:
#         #     out.writelines(modified_lines)
        
#         # Set up folders
#         user_home = os.path.expanduser("~")
#         desktop_path = os.path.join(user_home, "Desktop")
#         results_root = os.path.join(desktop_path, "Results")
#         results_folder = os.path.join(results_root, folder)
#         os.makedirs(results_folder, exist_ok=True)

#         # Run simulation
#         try:
#             subprocess.run(["bsimw32", filename, prefix, "wait=0"], check=True, timeout=60)
#         except subprocess.TimeoutExpired:
#             print("RSoft timed out")
#         # Read .mon file (you might want to standardize naming on output too)
#         uf.read(f"{name_tag}.mon")
#         x, y = uf.get_arrays()

#         # Save results
#         results = {0: y}  # just one run per call
#         with open("Throughput_results.csv", mode="w", newline="") as file:
#             writer = csv.writer(file)
#             writer.writerow(["x", name_tag])
#             for i in range(len(x)):
#                 writer.writerow([x[i], y[i]])

#         # Move files to results folder
#         for file in os.listdir():
#             if file.startswith(name_tag) or file == filename:
#                 shutil.move(file, os.path.join(results_folder, file))

#     elif skip:
#         warnings.warn('Prior space will change, use plot only for validation purposes!')
#     ###########################################################################################

#     df = pd.read_csv("Throughput_results.csv")
#     average = df[name_tag].mean()
#     return -average

# def mp_eval(params):
#     print(f"[PID {os.getpid()}] Starting RunRsoft with {params}")
#     return RunRsoft(params, skip=False)

# opt = Optimizer(
#     dimensions = para_space,
#     base_estimator = "GP",
#     acq_func = "EI",
#     random_state=42
# )

# if __name__ == "__main__":    
#     total_calls = 30
#     batch_size = 5
#     all_results = []

#     for i in range(0, total_calls, batch_size):
#         print(f"\n Batch {i//batch_size + 1}")
        
#         # Suggest next batch of points
#         param_batch = opt.ask(batch_size)

#         # Evaluate in parallel
#         with mp.Pool(batch_size) as pool:
#             result_batch = pool.map(mp_eval, param_batch)

#         # Feed results back to optimizer
#         opt.tell(param_batch, result_batch)
        
#         all_results.extend(zip(param_batch, result_batch))
#         # dump(opt, "rsoft_opt_checkpoint.pkl", store_objective=False)

In [8]:
# import matplotlib.pyplot as plt
# import numpy as np
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# # Unpack your results
# x_iters = [r[0] for r in all_results]  # parameter sets
# y_vals = [r[1] for r in all_results]  # throughput values (already positive if returned directly)

# c_num = np.arange(len(x_iters))  # iteration counter
# param_names = [dim.name for dim in opt.space.dimensions]
# n_params = len(param_names)

# # Find best result
# best_idx = np.argmax(y_vals)
# best_params = x_iters[best_idx]
# best_throughput = y_vals[best_idx]

# # Create plots
# fig, axes = plt.subplots(n_params, 1, figsize=(8, 4.5 + 1.5 * n_params), sharex=False)

# if n_params == 1:
#     axes = [axes]

# for i, (param_name, ax) in enumerate(zip(param_names, axes)):
#     x_vals = [x[i] for x in x_iters]

#     scatter = ax.scatter(x_vals, y_vals, c=c_num, cmap='viridis_r', s=60, edgecolor='k', label="Evaluations")
#     ax.scatter(best_params[i], best_throughput, c='red', s=100, label="Best", zorder=3, edgecolor='black')
    
#     ax.set_ylabel("Throughput", fontsize=12)
#     ax.set_xlabel(param_name, fontsize=12)
#     ax.set_title(f"{param_name} vs Throughput", fontsize=14)
#     ax.grid(True, linestyle='--', alpha=0.5)
#     ax.legend()

#     # Add colorbar
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes("right", size="4%", pad=0.05)
#     cbar = plt.colorbar(scatter, cax=cax)
#     cbar.set_label("Iteration")

# plt.tight_layout()
# plt.show()

# # Print best parameters
# print("Best Parameters:")
# for name, val in zip(param_names, best_params):
#     print(f"  {name}: {val:.4f}")
# print(f"Max Throughput: {best_throughput:.4f}")


In [9]:
# if __name__ == "__main__":
#     rsoft_result = gp_minimize(RunRsoft, 
#                            para_space,
#                            n_calls = 100,
#                            n_initial_points= 10,
#                            acq_func = "EI",
#                            acq_optimizer = "lbfgs",
#                            random_state=42,
#                            n_jobs = 4
#                            )

In [10]:
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# x_iters = rsoft_result.x_iters 
# y_vals = [-val for val in rsoft_result.func_vals]
# c_num = np.arange(len(x_iters))

# param_names = [dim.name for dim in rsoft_result.space.dimensions]
# n_params = len(param_names)

# best_params = rsoft_result.x
# best_throughput = -rsoft_result.fun

# fig, axes = plt.subplots(n_params, 1, figsize=(8, 4.5 + 1.5 * n_params), sharex=False)

# if n_params == 1:
#     axes = [axes]  # ensure axes is always a list

# for i, (param_name, ax) in enumerate(zip(param_names, axes)):
#     x_vals = [x[i] for x in x_iters]
    
#     scatter = ax.scatter(x_vals, y_vals, c=c_num, cmap='viridis_r', s=60, edgecolor='k')
#     ax.scatter(best_params[i], best_throughput, c='red', s=100, label="Best", zorder=3)
    
#     ax.set_ylabel("Throughput")
#     ax.set_xlabel(param_name)
#     ax.set_title(f"{param_name} vs Throughput")
#     ax.grid(True, linestyle='--', alpha=0.5)
#     ax.legend()

#     plt.colorbar(scatter, ax=ax, label="Iteration")
# plt.tight_layout()
# plt.show()

In [11]:
# print("Best parameters:")
# for name, val in zip(param_names, best_params):
#     print(f"  {name}: {val:.4f}")

# print(f"Max throughput: {best_throughput:.4f}")

In [12]:
# Corediam_result = [8.2051, 8.2027]
# Length_result = [1000.0000, 10.0000]
# Core_index_result = [1.8904, 1.5414]
# Max_corediam_prior_lim = [10, 20]

# df = pd.DataFrame({
#     "Max Corediam Prior Limit": Max_corediam_prior_lim,
#     "Best Corediam": Corediam_result,
#     "Best Length": Length_result,
#     "Best Core Index": Core_index_result
# })

# std_row = pd.DataFrame({
#     "Max Corediam Prior Limit": ["STD"],
#     "Best Corediam": [np.std(Corediam_result)],
#     "Best Length": [np.std(Length_result)],
#     "Best Core Index": [np.std(Core_index_result)]
# })

# df = pd.concat([df, std_row], ignore_index=True)


# print(df.to_string(index=False))